In [2]:
from pymongo import MongoClient
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [3]:
def get_historical_data(symbol: str, last_n: int):
    try:
        # Connect to MongoDB
        client = MongoClient("mongodb://127.0.0.1:27017/")
        db = client["stockdata"]
        collection = db[symbol]

        # Retrieve the last N documents for the given symbol
        cursor = collection.find().sort("_id", -1).limit(last_n)
        data_list = list(cursor)

        # Convert the selected documents into a DataFrame
        df = pd.DataFrame(data_list)

        return df

    finally:
        # Close the MongoDB connection
        client.close()

In [4]:
df = get_historical_data('TURSG.IS',50)

In [5]:
df.head()

,_id,Open,High,Low,Close,Adj Close,Volume
0,2024-02-28,48.000000,48.540001,47.020000,47.900002,47.900002,1650077
1,2024-02-27,49.639999,49.680000,47.840000,47.919998,47.919998,2800885
2,2024-02-26,50.349998,50.799999,49.580002,49.580002,49.580002,2175003
3,2024-02-23,50.650002,51.250000,49.000000,50.349998,50.349998,5101495
4,2024-02-22,51.549999,52.200001,50.250000,50.549999,50.549999,4381130


In [15]:
import plotly.express as px
from pymongo import MongoClient
import pandas as pd

def plot_stock(symbol, last_n):
    try:
        client = MongoClient("mongodb://127.0.0.1:27017/")
        db = client["stockdata"]
        collection = db[symbol]

        cursor = collection.find().sort("_id", -1).limit(last_n)
        data_list = list(cursor)

        df = pd.DataFrame(data_list)

        df["_id"] = pd.to_datetime(df["_id"])

        fig = px.line(df, x="_id", y=["Open", "High", "Low", "Close"], 
                      labels={"_id": "Date", "value": "Price"},
                      title=f"Stock Price Over Time {symbol}")

        fig.update_layout(
            xaxis_title="Date",
            yaxis_title="Price",
            legend_title="Type",
            hovermode="x",
            template="plotly_dark",
            width = 1500,
            height = 750
        )

        fig.show()

    finally:
        client.close()



In [16]:
plot_stock('CRFSA.IS',21)